<a href="https://colab.research.google.com/github/linghduoduo/Deep-Learning/blob/master/PyTorchCookBook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import collections
import os
import shutil
import tqdm

In [2]:
import numpy as np
import PIL.Image
import torch
import torchvision

In [3]:
torch.__version__
torch.version.cuda
torch.backends.cudnn.version()
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

固定随机种子

In [5]:
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

指定程序运行在特定 GPU 卡上 在命令行指定环境变量

CUDA_VISIBLE_DEVICES=0,1 python train.py

指定程序运行在特定  在代码中指定

os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

判断是否有 CUDA 支持

In [6]:
torch.cuda.is_available()

True

设置为 cuDNN benchmark 模式.  Benchmark 模式会提升计算速度，但是由于计算中有随机性，每次网络前馈结果略有差异。

In [8]:
torch.backends.cudnn.benchmark = True

如果想要避免这种结果波动，设置

In [11]:
torch.backends.cudnn.deterministic = True

清除 GPU 存储 - 有时 Control-C 中止运行后 GPU 存储没有及时释放，需要手动清空。在 PyTorch 内部可以

In [12]:
torch.cuda.empty_cache()

或在命令行可以先使用 ps 找到程序的 PID，再使用 kill 结束该进程

ps aux | grep pythonkill -9 [pid]

或者直接重置没有被清空的 GPU

nvidia-smi --gpu-reset -i [gpu_id]

张量基本信息

In [18]:
a = torch.tensor([1,2,3,4,5])
a.type()   # Data type

'torch.LongTensor'

In [19]:
a.size()   # Shape of the tensor. It is a subclass of Python tuple

torch.Size([5])

In [20]:
a.dim()    # Number of dimensions.

1

数据类型转换

In [23]:
# Set default tensor type. Float in PyTorch is much faster than double.
torch.set_default_tensor_type(torch.FloatTensor)

tensor = torch.rand(10)

# Type convertions.
tensor = tensor.cuda()
tensor = tensor.cpu()
tensor = tensor.float()
tensor = tensor.long()


torch.Tensor 与 np.ndarray 转换

In [26]:
# torch.Tensor -> np.ndarray.
ndarray = tensor.cpu().numpy()

# np.ndarray -> torch.Tensor.
tensor = torch.from_numpy(ndarray).float()
tensor = torch.from_numpy(ndarray.copy()).float()  # If ndarray has negative stride